# CNN

In [10]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform

In [11]:
class CNN(nn.Module):  
    def __init__(self):  
        super(CNN, self).__init__()  
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)  
        self.relu1 = nn.ReLU()  
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)  
        self.relu2 = nn.ReLU()  
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  

        self.fc1 = nn.Linear(32 * 8 * 8, 128)  
        self.relu3 = nn.ReLU()  
        self.fc2 = nn.Linear(128, 10)  

    def forward(self, x):  
        x = self.pool1(self.relu1(self.conv1(x)))  
        x = self.pool2(self.relu2(self.conv2(x)))  
        x = x.view(x.size(0), -1)  
        x = self.relu3(self.fc1(x))  
        x = self.fc2(x)  
        return x

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


100%|██████████| 170M/170M [02:08<00:00, 1.32MB/s] 


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}")


Epoch 1/5, Loss: 1.349057561223963
Epoch 2/5, Loss: 1.0036389993614518
Epoch 3/5, Loss: 0.8664042705613989
Epoch 4/5, Loss: 0.7674917323575596
Epoch 5/5, Loss: 0.6847999383452911


In [14]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 69.28%


In [15]:
# Save
torch.save(model.state_dict(), 'cnn_model.pth')

# Load
model.load_state_dict(torch.load('cnn_model.pth'))
model.eval()


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)